## ИМПОРТЫ

In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=a6b537b8c8b4b65a23ac39b160a71c3dba9f1783412f621f863eeae82d795dac
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math as mt

import datetime
import warnings
import shutil
import time
import os

# from optbinning import BinningProcess
from datetime import datetime as dt
from datetime import timedelta
from tqdm import tqdm


sns.set(style="darkgrid", font_scale=1.4)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

# from libro.BruteFormater import BruteData as bf
# from libro import LoadMaster as lm
# from libro import EasyWorker as ew

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, RobustScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import roc_auc_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.ensemble import IsolationForest, RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neural_network import MLPRegressor
from sklearn import metrics, model_selection
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.svm import SVR

from xgboost import XGBRegressor

import torch
# from torch.utils.data import TensorDataset, DataLoader
# from torch.nn import functional as F
# from torch.autograd import Variable
# from torch import nn

from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense


import statsmodels.api as sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model = ['audi', 'bmw', 'ford', 'hyundi', 'merc', 'skoda', 'toyota', 'vauxhall', 'vw']
model_n = ['Audi', 'BMW', 'Ford', 'Hyundai', 'Mercedes', 'Skoda', 'Toyota', 'Opel', 'Volkswagen'] # Массив для переименования марок
models = model.copy()
# model = ['focus', 'cclass', unclean_cclass', 'unclean_focus']

for i in range(len(model)): # загрузка данных
    path = r'/content/drive/My Drive/ProjectPIVO/' + str(model[i]) + '.csv' 
    models[i] = pd.read_csv(path)
    # Переименовываем марки автомобиля
    models[i]['Brand'] = f'{model_n[i]}'
    # Переименование колонок на Uppercase 
    models[i] = models[i].rename(columns = {'model' : 'Model', 'year' : 'Year', 'price' : 'Price', 
        'transmission' : 'Transmission', 'mileage' : 'Mileage', 'fuelType' : 'Fuel_Type', 'tax' : 'Tax', 'mpg' : 'MPG', 
        'engineSize' : 'Engine_Size', 'tax(£)' : 'Tax2'})
all_models = pd.DataFrame() # Создание датафрэйма
# Добавление содержимого всех csv файлов в общий датафрейм all_models
for i in range(len(model)):
    models[i]['Brand'] = f'{model_n[i]}' 
    all_models = pd.concat([all_models, models[i]]) 

##Первичная первичная обработка

In [ ]:
# Чистим записи, содержащие все пустые значения (таковых нет)
all_models = all_models.dropna(how='all')
# Мерджим Tax с Tax2, если у первого пропущены значения
print(len(all_models[all_models.Tax.isna()]))
all_models.Tax.fillna(all_models.Tax2, inplace = True)
print(len(all_models[all_models.Tax.isna()]))
print(all_models[all_models.Tax2.isna() != True])
# Удаляем Tax2
all_models = all_models.drop('Tax2', axis = 1)
# Вычисляем возраст автомобиля
all_models['Car_age'] = 2022 - all_models['Year']
# Удаляем колонку года выпуска автомобиля
all_models = all_models.drop('Year', axis = 1)
all_models = all_models.reset_index(drop = True)

In [ ]:
# Удаление пробелов в тексте
all_models['Model'] = all_models['Model'].apply(lambda x: x.strip()) 

# Выделение данных моделей автомобилей в класс электрических
elec = ['Ioniq', 'i3', 'Ampera'] # перенос данных марок авто в класс электромобилей
all_models.loc[all_models.Model.isin(elec), 'Fuel_Type'] = 'Electric'

# # Проверка столбца года выпуска на битые значения
f_year_1 = all_models.Car_age < 50
f_year_2 = all_models.Car_age > -1
print(f'Записей с некорректным годом выпуска : {(~f_year_1).sum(), (~f_year_2).sum()}') # ПРОВЕРКА СТОЛБЦА ГОД ВЫПУСКА НА БИТЫЕ ЗНАЧЕНИЯ

f_engine = all_models.Engine_Size != 0
print(f'Записей с некорректным объёмом двигателя : {(~f_engine).sum()}')

# Электрические + некорректный объем двигателя
# print(len(all_models[~f_engine][all_models.Fuel_Type == 'Electric']))

ggg = all_models[all_models.Fuel_Type == 'Electric']

f_electric = all_models.Fuel_Type == 'Electric'
print(f'Записей электромобилей : {(f_electric).sum()}')

# # Удаление битых данных
all_models = all_models[f_year_1]
print(len(all_models))
all_models = all_models[f_year_2]
print(len(all_models))
all_models = all_models[f_engine]
print(len(all_models))
all_models = all_models[~f_electric]
print(len(all_models))
all_models.Tax = all_models.Tax.astype(int)
print(len(all_models))
all_models = all_models[all_models.Transmission != 'Other']
print(len(all_models))
all_models = all_models[all_models.Fuel_Type != 'Other']
print(len(all_models))
all_models = shuffle(all_models).reset_index(drop=True)
print(len(all_models))

In [ ]:
# Визуальный вывод данных по столбцам
sns.set(rc = {'figure.figsize':(12,8)})
columnNames = {
    'Price': 'Цена',
    'Mileage': 'Пробег',
    'MPG': 'Расход топлива',
    'Car_age': 'Возраст',
    'Engine_Size': 'Объем двигателя',
    'Tax': 'Налог',
}

for name in columnNames:
  plt.hist(all_models[name], bins=50)
  plt.ylabel('Количество автомобилей')
  plt.xlabel(columnNames[name])
  plt.show()

In [ ]:
for i in all_models.columns: # статистика переменных типа "объект"
    if all_models[i].dtype == 'O':
        display(all_models[i].value_counts().to_frame())

##Постобработка

### Создание dummy переменных

In [ ]:
all_models_d = pd.get_dummies(all_models)

### Отбор значений с ненулевыми значениями более 1%

In [ ]:
f_f1 = ((all_models_d == 0).sum()/(len(all_models_d)) < 0.99)
all_models_d = all_models_d.loc[:, f_f1]

### Матрица корреляций и удаление мультиколлинеарных переменных

In [ ]:
sns.set(rc = {'figure.figsize':(8,6)})
sns.heatmap(all_models_d.iloc[:, :6].corr(),
            annot = True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', cbar=False)

In [ ]:
df_corr = all_models_d.corr() # Определение столбцов с взаимной корреляцией больше 0,7
df_bad = pd.DataFrame()
for i in df_corr.columns:
    for j in df_corr.index:
        in_col = sorted([i, j])
        if ((abs(df_corr.loc[in_col[0], in_col[1]]) > 0.7) & (in_col[0] != in_col[1])):
            df_bad = df_bad.append(pd.DataFrame([in_col[0], in_col[1], df_corr.loc[in_col[0], in_col[1]]]).T)
df_bad.columns = [0, 1, 'Value']
df_bad = df_bad.drop_duplicates().reset_index(drop = True)

In [ ]:
col_to_drop = []
for i in range(len(df_bad)): # Определяем столбец, который дает меньший вес прогноз. значению (цене)
    x = abs(df_corr.loc['Price', [df_bad.iloc[i, 0], df_bad.iloc[i, 1]]])
    print(1111, x)
    print(2222, np.argmin(x))
    # col_to_drop.append(df_bad.loc[i, np.argmin(x)])

In [ ]:
all_models_d = all_models_d.drop(col_to_drop, axis=1) # Удаляем столбец, который выбрали

In [ ]:
def std_df(df, col): # Функция стандартизации
    print(777, df, col)
    var = mt.sqrt(df[col].var())
    mean = df[col].mean()
    print(888, (df[col] - mean) / var )
    return (df[col] - mean) / var 
def un_std_df(df, col): # НЕ ИСПОЛЬЗУЕТСЯ
    var = mt.sqrt(all_models[col].var())
    mean = all_models[col].mean()
    return (df[col] * var) + mean
def un_std_per(per): # НЕ ИСПОЛЬЗУЕТСЯ
    var = mt.sqrt(all_models.Price.var())
    mean = all_models.Price.mean()
    return (per * var) + mean

### Отбор переменных с корреляций больше 0,05 по модулю

In [ ]:
all_models_d1 = all_models_d.copy()

list_corr = all_models_d.corr().iloc[:, 0] # Определяем столбцы с низкой коррреляцией чтобы потом их выкинуть
f_f2 = (abs(list_corr) > 0.05)
all_models_d = all_models_d.loc[:, f_f2]

### Стандартизация переменных

In [ ]:
colls = ['Mileage', 'Tax', 'MPG', 'Engine_Size', 'Car_age']
all_models_d_std = all_models_d.copy()
for i in colls:
    try:
      all_models.var()
        all_models_d_std[i] = std_df(all_models_d, i)
    except: pass

In [ ]:
all_models_d_std # Вывод финальной таблицы с которой мы пойдем в нейронку

# Построение нейронной сети

In [ ]:
all_models_d_std = all_models_d_std[:5000] #для проверки

In [ ]:
def get_score(y_pred, y_test):
    """
      Принимает на вход предсказанные значения y и тестовые значения y
      Возвращает массив:
        0: R2 - Коэффициент детерминации
        1: MAE - средняя абсолютная ошибка
        2: RMSE - среднеквадратичная ошибка
        3: MAPE - средняя абсолютная ошибка. В среднем ошибаемся на MAPE %
    """
    # Получаем коэффициент детерминации и округляем до 5 знаков
    R2 = round(r2_score(list(y_test), y_pred), 5)
    # Вычисление средней абсолютной ошибки
    MAE = int(mean_absolute_error(list(y_test), y_pred))
    # Вычисление среднеквадратичной ошибки
    RMSE = int(mean_squared_error(list(y_test), y_pred, squared=False))
    # Вычисление средней абсолютной ошибки в процентах и округление до 5 знаков
    MAPE = round(mean_absolute_percentage_error(list(y_test), y_pred), 5)

    return [R2, MAE, RMSE, MAPE]

In [ ]:
len_x = len(list(X_test))
len_y = 1
def baseline_model():
	"""
		Возвращает нейронную сеть из 6 слоев:
			1: Входной слой размерности 5/4 * dataset_length. Экспоненциальная функция активации
			2: Скрытый слой размерности dataset_length. Сигмоидная функция активации
			3: Скрытый слой размерности 3/4 * dataset_length. ReLu функция активации
			4: Скрытый слой размерности 2/4 * dataset_length. ReLu функция активации
			5: Скрытый слой размерности 1/4 * dataset_length. ReLu функция активации
			6: Выходной слой размерности 1. Линейная функция активации
	"""
	# Создаем sequential модель через библиотеку keras
	model = Sequential()
 
	# Добавление слоев.
	# Количество нейронов = (длина выборки * (5 - i)) / 4,
	# где i - порядковый номер слоя

	# Входной слой
	# Передаем размер входного слоя input_dim = размеру X-тестовой выборки. В качестве функции активации - экспоненциальная.
	model.add(Dense(5*len_x/4, input_dim=len_x, kernel_initializer='normal', activation='exponential'))
 
	# Скрытые слои
	# Передаем сигмоидную функцию активации. A = (1 / (1 + e^(-x)))
	model.add(Dense(4*len_x/4, kernel_initializer='normal', activation='sigmoid'))
 
	# Передаем ReLu функцию активации. A(x) = max(0, x)
	model.add(Dense(3*len_x/4, kernel_initializer='normal', activation='relu')) #линейная
	model.add(Dense(2*len_x/4, kernel_initializer='normal', activation='relu'))
	model.add(Dense(len_x/4, kernel_initializer='normal', activation='relu'))
 
	# Выходной слой
	# Размерность = 1. По умолчанию в качестве функции активации - линейная
	model.add(Dense(len_y, kernel_initializer='normal'))
	
	# Настройка модели для обучения.
	# В качестве функции потери - MSE, среднеквадратичная ошибка
	# В качестве оптимизатора - алгоритм Адама
	model.compile(loss='mean_squared_error', optimizer='adam')
 	return model

# Указываем число для рандомизации random_state
seed = 322
# Количество эпох обучения
epochs = 5
# Количество выборок для обновления градиента (для подгонки и прогнозирования)
batch_size = 2

# Для регрессионной оценки использован KerasRegressor
estimator = KerasRegressor(build_fn=baseline_model, epochs=epochs, batch_size=batch_size, verbose=0)

# Проверка модели через cross-валидатор KFold.
# Переменная принимает индексы из тренировочных и тестовых выборок
kfold = KFold(n_splits=5, random_state=seed, shuffle=True)

# X - все столбцы, за исключением столбка цен
# y - столбец цен
X, y = all_models_d_std.drop(columns = ['Price']), all_models_d_std['Price']

predicts = []
for train_index, test_index in kfold.split(X):
		print(len(train_index), len(test_index))
		# Получение тренировочной и тестовой выборок из оригинального датафрейма
		X_train, X_test = X.loc[train_index, :], X.loc[test_index, :]
		y_train, y_test = y[train_index], y[test_index]
		# Подгонка модели
		estimator.fit(X_train, y_train)
		# Прогноз на тестовой выборке данных
		prediction = estimator.predict(X_test)
		# Добавляем текущий прогноз в массив
		predicts.append(prediction)
		# Вывод R2, MAE, RMSE, MAPE
		print(get_score(prediction, y_test))